In [1]:
import os
import random
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D


In [2]:
# Set your local data directory here.
# Assuming you have folders like 'apple', 'banana', 'orange'
# inside 'data_dir', and each of those has 'fresh' and 'rotten' subfolders.
DATA_DIR = 'C:/Users/Akarshjayanth.M.Naik/Desktop/my_fruit_checker/data/training' # Replace with your actual path

CATEGORIES = [] # List to hold combined categories (e.g., 'fresh_apple', 'rotten_banana')
FRUIT_TYPES = [] # List to hold fruit types (apple, banana, orange) - dynamically detected
IMG_SIZE = 50 # Image size for resizing

In [3]:
def create_training_data():
    training_data = []
    category_index = 0 # Initialize index for combined categories
    for fruit_type in FRUIT_TYPES: # Iterate through fruit type folders (apple, banana, etc.)
        for quality in ['fresh', 'rotten']: # Iterate through 'fresh' and 'rotten' subfolders
            combined_category_name = f"{quality}_{fruit_type}" # e.g., 'fresh_apple'
            CATEGORIES.append(combined_category_name) # Add combined category to list
            path = os.path.join(DATA_DIR, fruit_type, quality) # Path to fruit_type/quality folder
            class_num = category_index # Use current category_index as class number
            for img in os.listdir(path): # Iterate through images in the folder
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) # Load in grayscale
                    if img_array is None: # Check if image was loaded successfully
                        continue
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # Resize to IMG_SIZE x IMG_SIZE
                    training_data.append([new_array, class_num]) # Add image and label
                except Exception as e:
                    print(f"Error processing image: {img} in category: {combined_category_name}")
                    print(e)
                    pass
            category_index += 1 # Increment category index for the next combined category
    return training_data

In [4]:
# Dynamically detect fruit types (subfolders in DATA_DIR)
for item in os.listdir(DATA_DIR):
    fruit_type_path = os.path.join(DATA_DIR, item)
    if os.path.isdir(fruit_type_path):
        # Check if it contains 'fresh' and 'rotten' subfolders (basic validation)
        if os.path.isdir(os.path.join(fruit_type_path, 'fresh')) and os.path.isdir(os.path.join(fruit_type_path, 'rotten')):
            FRUIT_TYPES.append(item)

print("Detected Fruit Types:", FRUIT_TYPES)

if not FRUIT_TYPES:
    raise ValueError(f"No fruit type folders with 'fresh' and 'rotten' subfolders found in '{DATA_DIR}'. Please check your data directory structure.")

training_data = create_training_data()

print("Combined Categories (for model output):", CATEGORIES)
print("Number of Combined Categories:", len(CATEGORIES))

random.shuffle(training_data)

X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

X = X/255.0

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Detected Fruit Types: ['apple', 'banana', 'orange']
Combined Categories (for model output): ['fresh_apple', 'rotten_apple', 'fresh_banana', 'rotten_banana', 'fresh_orange', 'rotten_orange']
Number of Combined Categories: 6
Shape of X: (14133, 50, 50, 1)
Shape of y: (14133,)


In [5]:

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128))
model.add(Dropout(0.25))

# Output layer now matches the number of combined categories
model.add(Dense(len(CATEGORIES))) # Output layer size is number of combined categories
model.add(Activation('softmax')) # Softmax for multi-class

model.compile(loss='sparse_categorical_crossentropy', # Sparse categorical crossentropy for integer labels
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=32, validation_split=0.3)

model.save('fruitquality-structured-cnn.h5')
print("Model saved as fruitquality-structured-cnn (h5)")

c:\Users\Akarshjayanth.M.Naik\Desktop\my_fruit_checker\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.4893 - loss: 1.2782 - val_accuracy: 0.7380 - val_loss: 0.7246
Epoch 2/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.7216 - loss: 0.7407 - val_accuracy: 0.7285 - val_loss: 0.6583
Epoch 3/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.7864 - loss: 0.5659 - val_accuracy: 0.8394 - val_loss: 0.4530
Epoch 4/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.8228 - loss: 0.4777 - val_accuracy: 0.8017 - val_loss: 0.5281
Epoch 5/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8401 - loss: 0.4246 - val_accuracy: 0.8677 - val_loss: 0.3570
Epoch 6/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8734 - loss: 0.3371 - val_accuracy: 0.8795 - val_loss: 0.3424
Epoch 7/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.8893 - loss: 0.3083 - val_accuracy: 0.8660 - val_loss: 0.3492
Epoch 8/32
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.8916 - loss: 0.3011 - va

Model saved as fruitquality-structured-cnn (h5)
